# Was sind die größten und komplexesten Methoden?

## 1. Verbindung zur Datenbank
Es wird eine Verbindung zur Neo4j-Datenbank aufgebaut.

In [2]:
import py2neo

graph = py2neo.Graph(bolt=True, host='localhost', user='neo4j', password='neo4j')

## 2. Cypher-Abfrage
Es wird eine Abfrage an die Datenbank gestellt. Das Ergebnis wird in einem Dataframe (pandas) gespeichert.

In [19]:
import pandas as pd

query ="MATCH (t:Type)-[:HAS_SOURCE]->(f), (t)-[:DECLARES]->(m:Method) RETURN t.fqn as type, m.signature as signature,  sum(m.effectiveLineCount) as loc, sum(m.cyclomaticComplexity) as complexity"
df = pd.DataFrame(graph.run(query).data())


## 3. Datenaufbereitung
Zur Kontrolle werden die ersten fünf Zeilen des Ergebnisses der Abfrage als Tabelle ausgegeben.

In [20]:
df.head()

,complexity,loc,signature,type
0,1,2,void <init>(),org.junit.rules.TestWatchmanTest$FailingTest
1,2,2,java.lang.Class getType(),org.junit.experimental.theories.ParameterSigna...
2,1,2,void forgetMax(),org.junit.tests.experimental.max.JUnit38Sortin...
3,0,0,void run(junit.framework.TestResult),junit.tests.extensions.ExtensionTest$TornDown
4,1,6,void doesNotConsiderParametersWhichChangedAfte...,org.junit.runners.parameterized.TestWithParame...


Im folgenden Codeabschnitt wird das Dataframe absteigend entsprechend der Anzahl der Quelltextzeilen (loc) sortiert und die ersten zehn Methoden extrahiert. Die Werte werden in den Variablen <font face="Courier">x_loc</font> und <font face="Courier">y_method_loc</font> gespeichert.

In [14]:
# zu implementieren


Im folgenden Codeabschnitt wird das Dataframe absteigend entsprechend der Komplexität (complexity) sortiert und die ersten zehn Methoden extrahiert. Die Werte werden in den Variablen <font face="Courier">x_complexity</font> und <font face="Courier">y_method_complexity</font> gespeichert.

In [24]:
# zu implementieren

## 4. Visualisierung
Die Daten werden mittels eines Bar Charts visualisiert (https://plot.ly/python/horizontal-bar-charts/, x=x_loc, y=y_method_loc).

In [25]:
# zu implementieren

Die Daten werden mittels eines Bar Charts visualisiert (https://plot.ly/python/horizontal-bar-charts/, x=x_complexity, y=y_method_complexity).

In [ ]:
# zu implementieren